In [18]:
%run 2.tool-calling.ipynb

# Create Simple Graph

In [19]:
from langgraph.graph import StateGraph, START, END

In [20]:
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]

In [21]:
tool_list = [search]

In [22]:
from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            tool_call_id = tool_call["id"]
            tool_result = self.tools_by_name[tool_name].invoke(tool_args)
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_name,
                    tool_call_id=tool_call_id,
                )
            )
        return {"messages": outputs}


def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


In [23]:
graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = BasicToolNode(tools=tool_list)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", END: END},
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")


graph = graph_builder.compile()

### See it in action

In [33]:
from uuid import uuid4
from sys import exit
from langgraph.types import Command
import json


user_id = uuid4().hex

config = {"configurable": {"thread_id": user_id}}
# TODO: Other metadata?


try:
    user_input = input("[User]: ")

    for event in graph.stream(
        {"messages": [{"role": "user", "content": user_input}]}, config=config
    ):
        
        # Check for interrupt
        if "__interrupt__" in event:
            interrupt_obj = event["__interrupt__"][0]
            # Usually a list with one Interrupt
            prompt = interrupt_obj.value  # The payload sent to human
            user_query = prompt["query"]
            print(f"[System]: [User] asks {user_query}")

            # Get human input to resume
            human_response = input("[Support]: ")

            # Resume the graph with human input
            resume_events = graph.stream(Command(resume=human_response), config=config)
            for resume_event in resume_events:
                # Process resumed events (messages, tools, etc.)
                print(resume_event)
            break  # Exit current event loop to wait for next user input

        # Handle chatbot node (tool calls and messages)
        if "chatbot" in event:
            for message in event["chatbot"]["messages"]:
                # Check for tool calls
                if hasattr(message, "tool_calls") and message.tool_calls:
                    print("Tool usage:")
                    for tool_call in message.tool_calls:
                        tool_name = tool_call["name"]
                        tool_args = tool_call["args"]
                        tool_id = tool_call["id"]
                        print(f"{tool_id} | {tool_name} | {tool_args}")
                # Print chatbot message content if no tool calls
                elif message.content:
                    print("Message output:")
                    print(message.content)

        # Handle tools node (tool execution results)
        if "tools" in event:
            for message in event["tools"]["messages"]:
                print("Tool output:")
                tool_call_id = message.tool_call_id
                message_id = message.id
                tool_name = message.name
                content_preview = message.content[
                    :1000
                ]  # Limit content to 100 characters
                print(
                    f"{tool_call_id} | {message_id} | {tool_name} | {content_preview}"
                )
except KeyboardInterrupt:
    print("Exiting...")
    exit(0)


[User]:  What are the main categories and functions of the eight parts of speech in English grammar?


Tool usage:
call_1omae8o8 | duckduckgo_search | {'query': 'English grammar parts of speech'}
Tool output:
call_1omae8o8 | 4d4c3307-3adc-48e8-a93a-7de42f35a68c | duckduckgo_search | "The eight parts of speech are nouns, pronouns, adjectives, verbs, adverbs, prepositions, conjunctions, and interjections. Most words in English can be classified as one of these eight parts of speech, although some, like determiners, are categorized separately in modern grammar. The term part of speech refers to a word's role in a sentence. Parts of Speech English Grammar The parts of speech explain how a word is used in a sentence. There are eight main parts of speech (also known as word classes): nouns, pronouns, adjectives, verbs, adverbs, prepositions, conjunctions and interjections. Most parts of speech can be divided into sub-classes. What is a Parts of Speech? A part of speech is a category that describes the role a word plays in a sentence. These roles help you understand how words function in gramm